In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
import seaborn as sns
# Importations nécessaires pour dashboard
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

In [2]:
base_path = 'D:/PythonData/AdvancedPython/Projet/Données/3/'
dfs = {}  # Créer un dictionnaire vide pour stocker les DataFrames

for year in range(2008, 2022):  # Boucle de 2008 à 2021 inclus
    file_name = f"interventions{year}V3.xlsx"
    full_path = base_path + file_name
    dfs[year] = pd.read_excel(full_path)

# Après avoir exécuté cette boucle, chaque DataFrame sera accessible via dfs[année].
# Par exemple, pour accéder au DataFrame de 2008, utilisez dfs[2008].

In [3]:
years = list(dfs.keys())
totals = [df['Total interventions'].sum() for _, df in dfs.items()]
colors = ['#ccebc5', '#a8ddb5', '#7bccc4', '#4eb3d3', '#2b8cbe']
interventions = ['Incendies', 'Secours à personne', 'Accidents de circulation', 'Risques technologiques', 'Opérations diverses']



url = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson"
departements = gpd.read_file(url)

for year in range(2008, 2022):  # Cela parcourra de 2008 à 2021 inclus
    # Sélectionnez les données pour l'année actuelle
    df_carte = dfs[year]

    bspp_row = df_carte[df_carte['Numéro'] == 'BSPP'].copy().iloc[0]  # Obtenir la ligne BSPP

    new_rows = []
    for dep in ['75', '92', '93', '94']:
        new_row = bspp_row.copy()
        new_row['Numéro'] = dep
        new_rows.append(new_row)

    # Supprimer la ligne BSPP originale
    df_carte = df_carte[df_carte['Numéro'] != 'BSPP']

    # Ajouter les nouvelles lignes à la DataFrame
    df_carte = pd.concat([df_carte, pd.DataFrame(new_rows)], ignore_index=True)


    # Convertir les numéros de département en format avec zéros non significatifs
    df_carte['Numéro'] = df_carte['Numéro'].apply(lambda x: str(x).zfill(2))

    # Fusionner les données géographiques des départements avec vos données
    merged = departements.set_index('code').join(df_carte.set_index('Numéro'))

    # Exclure les départements 77, 78, 91 et 95
    merged = merged.drop(['77', '78', '91', '95'], errors='ignore')

    # Fusionner les données géographiques des départements avec vos données
    merged = departements.set_index('code').join(df_carte.set_index('Numéro'))



    # Réinitialiser l'index
    merged = merged.reset_index()
    print(merged[merged['code'] == 'BSPP'])

    # Calcul des quantiles pour l'échelle des couleurs
    quantiles = list(merged['Total interventions'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))

    # Créer la carte choroplèthe avec l'échelle ajustée
    m = folium.Map(location=[46.896242, 3.078600], zoom_start=5)
    folium.Choropleth(
        geo_data=departements,
        data=merged,
        columns=['code', 'Total interventions'],
        key_on='feature.properties.code',
        fill_color='GnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'Interventions par département {year}',
        threshold_scale=quantiles  # Utilisation de l'échelle basée sur les quantiles
    ).add_to(m)

    # Sauvegardez la carte dans un fichier HTML pour l'année en cours
    filename = f"map_{year}.html"
    m.save(filename)
    print(f"Carte sauvegardée pour l'année {year} dans le fichier {filename}")

In [20]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

with open('map_2021.html', 'r', encoding='utf-8') as f:
    map_content = f.read()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Filtre Année"),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(i), 'value': i} for i in range(2008, 2022)],
                value=2021
            ),
        ], width=3, lg=1.5)  # Réglage de la taille des colonnes pour un affichage réactif
        
    ], className='mb-1'),  # Ajout d'une marge en bas de la rangée

    dbc.Row([
            dbc.Col([
                dcc.Graph(id='bar-chart', style={'height': '50vh'}),  # Ajustement de la hauteur
            ], width=12, lg=6),
            dbc.Col([
                dcc.Graph(id='pie-chart', style={'height': '50vh'}),  # Ajustement de la hauteur
            ], width=12, lg=6),
    ], className='mb-1'),  # Ajout d'une marge en bas de la rangée

    dbc.Row([
        dbc.Col([
            html.Iframe(id='map-iframe', srcDoc=map_content, width='100%', height='370px'),  # Hauteur ajustée en pixels
        ], width=12, lg=6),
        dbc.Col([
            dcc.Graph(id='top5-bar-chart', style={'height': '50vh'}),  # Ajustement de la hauteur
        ], width=12, lg=6),
    ], className='mb-1'),

    dbc.Row([
            dbc.Col([
                dcc.Graph(id='hist-chart', style={'height': '50vh'}),  # Ajustement de la hauteur
            ], width=12, lg=6),
            dbc.Col([
                dcc.Graph(id='corre-chart', style={'height': '50vh'}),  # Ajustement de la hauteur
            ], width=12, lg=6),
    ], ),  # Ajout d'une marge en bas de la rangée
], fluid=True, style={'backgroundColor': '#E9ECEF'})  # Couleur de fond douce pour l'ensemble du container

In [21]:
# Graphique à barres pour l'évolution des interventions par année
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_bar_chart(selected_year):
   

    bar_fig = px.bar(
        x=years, y=totals, 
        labels={'x': 'Année', 'y': 'Nombre d\'interventions'}, 
        title='Évolution des interventions par année'
    )

    bar_fig.update_traces(marker_color='#83A1A4')  # Une couleur bleue standard pour les barres

    # Ajout des lignes pour chaque type d'intervention
    for idx, intervention in enumerate(interventions):
        yearly_totals = [df[intervention].sum() for _, df in dfs.items()]
        bar_fig.add_scatter(
            x=years, y=yearly_totals, 
            mode='lines', name=intervention, 
            line=dict(color=colors[idx])
        )

    bar_fig.update_layout(
        title={
            'text': 'Évolution des interventions par année',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=15),  # Taille de titre ajustée pour meilleure visibilité
        margin=dict(l=20, r=20, t=40, b=20)  # Marges réduites pour un meilleur ajustement dans l'espace alloué
    )

    return bar_fig

In [22]:
# Graphique à secteurs pour la répartition des interventions par type pour l'année sélectionnée
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_pie_chart(selected_year):
    répartition_interventions = dfs[selected_year][[
        'Incendies',
        'Secours à personne',
        'Accidents de circulation',
        'Risques technologiques',
        'Opérations diverses'
    ]].sum()

    pie_fig = px.pie(
        values=répartition_interventions.values, 
        names=répartition_interventions.index, 
        title=f"Répartition des interventions par type pour l'année {selected_year}",
        color_discrete_sequence=colors
    )

    pie_fig.update_layout(
        title={
            'text': f"Répartition des interventions par type pour l'année {selected_year}",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=15),  # Taille de titre ajustée pour meilleure visibilité
        margin=dict(l=20, r=20, t=40, b=20)  # Marges réduites pour un meilleur ajustement dans l'espace alloué
    )

    return pie_fig


In [23]:
@app.callback(
    Output(component_id='map-iframe', component_property='srcDoc'),
    Input(component_id='year-dropdown', component_property='value')
)
def update_map(selected_year):
    with open(f'map_{selected_year}.html', 'r', encoding='utf-8') as f:
        map_content = f.read()
    return map_content

In [24]:
@app.callback(
    Output('top5-bar-chart', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_top5_bar_chart(selected_year):
    df = dfs[selected_year]
    top5 = df.sort_values(by='Total interventions', ascending=False).head(5)

    bar_fig = px.bar(
        top5, 
        x='Département', 
        y='Total interventions',
        color='Département',
        title=f"Top 5 des départements avec le plus grand nombre d'interventions pour {selected_year}",
        color_discrete_sequence=colors
    )

    bar_fig.update_layout(
        title={
            'text': f"Top 5 des départements avec le plus grand nombre d'interventions pour {selected_year}",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        title_font=dict(size=15),  # Taille de titre ajustée pour une meilleure visibilité
        margin=dict(l=20, r=20, t=40, b=20),  # Marges réduites pour un meilleur ajustement
        
    )
    return bar_fig

In [25]:
@app.callback(
    Output('hist-chart', 'figure'), 
    [Input('year-dropdown', 'value')]
)
def update_histogram(selected_year):
    data_for_year = dfs[selected_year]
    fig = px.histogram(
        data_for_year, 
        x='Total interventions',
        nbins=30, 
        title=f"Histogramme des interventions pour l'année {selected_year}",
        color_discrete_sequence=['#4eb3d3']
    )

    fig.update_layout(
        xaxis_title="Nombre d'interventions",
        yaxis_title="Fréquence",
        margin=dict(l=20, r=20, t=40, b=20)
    )

    return fig

In [26]:
@app.callback(
    Output('corre-chart', 'figure'),  
    [Input('year-dropdown', 'value')]
)
def update_correlation_matrix(selected_year):
    data_for_year = dfs[selected_year]
    interventions_data = data_for_year[interventions]  # Assurez-vous que 'interventions' est une liste des colonnes pertinentes
    correlation_matrix = interventions_data.corr(method='pearson')
    transformed_correlation_matrix = np.arctan(correlation_matrix * (np.pi/2 - 0.1))

    fig = px.imshow(
        transformed_correlation_matrix, 
        text_auto=True,
        title=f"Matrice de Corrélation pour l'année {selected_year}",
        color_continuous_scale=colors

    )

    fig.update_layout(
        xaxis_title="Types d'interventions",
        yaxis_title="Types d'interventions",
        margin=dict(l=20, r=20, t=40, b=20)
    )

    return fig

In [27]:
# Lancer l'application
if __name__ == '__main__':
    app.run_server(debug=True)